In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_curve
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data1=pd.read_csv('/content/drive/MyDrive/dosdata/data1.csv')
data2=pd.read_csv('/content/drive/MyDrive/dosdata/data2.csv')
data3=pd.read_csv('/content/drive/MyDrive/dosdata/data3.csv')

train_data=pd.concat([data1,data2,data3]).reset_index(drop=True)

train_data = train_data.drop(columns='Unnamed: 0')

train_data["Label"] = train_data["Label"].replace(["Benign","Anomaly"],[0,1])
for label, index in zip(['Benign', 'Anomaly'], [0, 1]):
    print(f"{label}: {index}")

Mounted at /content/drive
Benign: 0
Anomaly: 1


In [ ]:
print(train_data["Label"].value_counts())

from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

X = train_data.drop("Label", axis=1)
y = train_data["Label"]

undersample = RandomUnderSampler(sampling_strategy='majority')
X, y = undersample.fit_resample(X, y)

print(Counter(y))

Label
0    1567950
1    1000448
Name: count, dtype: int64
Counter({0: 1000448, 1: 1000448})


In [ ]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

def optimize_sgd_classifier(X, y, cv=5):

    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    param_grid = {
        'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'max_iter': [100, 200, 300, 400, 500],
        'tol': [1e-3, 1e-4, 1e-5]
    }

    sgd = SGDClassifier(random_state=42)

    grid_search = GridSearchCV(
        estimator=sgd,
        param_grid=param_grid,
        cv=skf,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(X, y)

    print("Best Parameters:", grid_search.best_params_)
    print("Best Cross-Validation Score:", grid_search.best_score_)

    cv_results = pd.DataFrame(grid_search.cv_results_)
    cv_results = cv_results[['params', 'mean_test_score', 'std_test_score']]
    cv_results = cv_results.sort_values('mean_test_score', ascending=False).head()

    return grid_search.best_estimator_, cv_results

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_fold_performance(estimator, X, y, fold_idx, fold):
    train_idx, val_idx = fold
    X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
    y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

    scaler = StandardScaler()
    X_fold_train = scaler.fit_transform(X_fold_train)
    X_fold_val = scaler.transform(X_fold_val)

    estimator.fit(X_fold_train, y_fold_train)
    y_pred = estimator.predict(X_fold_val)
    y_pred_proba = estimator.decision_function(X_fold_val)

    metrics = {
        'fold': fold_idx + 1,
        'accuracy': accuracy_score(y_fold_val, y_pred),
        'precision': precision_score(y_fold_val, y_pred),
        'recall': recall_score(y_fold_val, y_pred),
        'f1': f1_score(y_fold_val, y_pred),
        'auc_roc': roc_auc_score(y_fold_val, y_pred_proba)
    }

    cm = confusion_matrix(y_fold_val, y_pred)
    metrics['tn'], metrics['fp'], metrics['fn'], metrics['tp'] = cm.ravel()

    return metrics, y_pred_proba, y_fold_val

def optimize_sgd_detailed(X, y, cv=5):
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    param_grid = {
        'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge'],
        'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
        'penalty': ['l2', 'l1', 'elasticnet'],
        'max_iter': [1000, 2000],
        'tol': [1e-3]
    }

    print("\nEvaluating SGD Classifier...")

    grid_search = GridSearchCV(
        estimator=SGDClassifier(random_state=42),
        param_grid=param_grid,
        cv=skf,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

    grid_search.fit(X_scaled, y)

    print(f"\nBest Parameters for SGD:", grid_search.best_params_)
    print(f"Best Cross-Validation Score: {grid_search.best_score_:.4f}")

    best_model = grid_search.best_estimator_
    fold_metrics = []
    fold_predictions = []
    fold_true_values = []

    for fold_idx, fold in enumerate(skf.split(X_scaled, y)):
        metrics, y_pred_proba, y_true = evaluate_fold_performance(
            best_model, X_scaled, y, fold_idx, fold
        )
        metrics['model_type'] = 'SGD'
        fold_metrics.append(metrics)
        fold_predictions.append(y_pred_proba)
        fold_true_values.append(y_true)

        print(f"\nFold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-Score: {metrics['f1']:.4f}")
        print(f"AUC-ROC: {metrics['auc_roc']:.4f}")
        print("\nConfusion Matrix:")
        print(f"TN: {metrics['tn']}, FP: {metrics['fp']}")
        print(f"FN: {metrics['fn']}, TP: {metrics['tp']}")

    plt.figure(figsize=(8, 6))
    for fold_idx in range(cv):
        fpr, tpr, _ = roc_curve(
            fold_true_values[fold_idx],
            fold_predictions[fold_idx]
        )
        plt.plot(
            fpr, tpr,
            label=f'Fold {fold_idx + 1} (AUC = {fold_metrics[fold_idx]["auc_roc"]:.4f})'
        )

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves for SGD Classifier (All Folds)')
    plt.legend()
    plt.show()

    metrics_df = pd.DataFrame(fold_metrics)

    print("\nAverage Metrics for SGD Classifier:")
    avg_metrics = metrics_df.groupby('model_type').agg({
        'accuracy': ['mean', 'std'],
        'precision': ['mean', 'std'],
        'recall': ['mean', 'std'],
        'f1': ['mean', 'std'],
        'auc_roc': ['mean', 'std']
    })
    print(avg_metrics)

    return best_model, grid_search.best_params_, metrics_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
best_model, best_params, metrics_df = optimize_sgd_detailed(X_train, y_train)

y_pred = best_model.predict(X_test)
print("\nFinal Test Set Performance:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix on Test Set')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()